## Install dependencies



In [ ]:
!pip install -r requirements.txt

## Upload data

Group images by separating each class into one folder then wrap all the folder another folder named `data`.

Ex.
```
data/ 
  │
  └─── class1/
  │        │
  |        └─── image1.png
  │        └─── image2.jpg
  |        └─── ...
  │   
  └─── class2/
  │        │
  |        └─── image123.png
  │        └─── image456.jpg
  |        └─── ...
  |
  └─── .../
           │
           └─── ...
```





## Remove unecessary files

Image file extension that are only acceptable are selected in the `image_exts`.

In [2]:
import cv2, imghdr, os

In [3]:
data_dir = 'data'
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [4]:
for image_class in os.listdir(data_dir):
  for image in os.listdir(os.path.join(data_dir, image_class)):
    image_path = os.path.join(data_dir, image_class, image)
    try:
      img = cv2.imread(image_path)
      tip = imghdr.what(image_path)
      if tip not in image_exts:
        print(f'Image not in ext list {image_path}')
        os.remove(image_path)
    except Exception as e:
            print(f'Issue with image {image_path}')

## Prepare, randomize, and normalize the images

Using the `tf.keras.utils.image_dataset_from_directory` with image_size of (256, 256).

See https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

In [5]:
import tensorflow as tf

In [6]:
data = tf.keras.utils.image_dataset_from_directory('data', image_size=(256, 256))
data = data.map(lambda x, y : (x/255, y)) # Normalize data between 0 and 1

Found 171 files belonging to 2 classes.


## Split the input and output pairs for training

Randomly split input and output pairs into sets of data: 70% for training, 20% for validation, and 10% for testing.

  - the training set is used to train the model
  - the validation set is used to measure how well the model is performing during training
  - the testing set is used to test the model after training

In [7]:
train_size = round(len(data)*0.7)
val_size = round(len(data)*0.2)
test_size = round(len(data)*0.1)
assert train_size + val_size + test_size == len(data), f'sum must be {len(data)}'

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

## Build & Train the Model

Build and train a [TensorFlow](https://www.tensorflow.org) model using the high-level [Keras](https://www.tensorflow.org/guide/keras) API.

### Build the neural network

In [8]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), 1, activation='relu', input_shape=(256, 256, 3)))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(32, (3, 3), 1, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(16, (3, 3), 1, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 16)       0

### Train

In [9]:
history = model.fit(train, epochs=20, validation_data=val)

Epoch 1/20
4/4 [==============================] - 9s 2s/step - loss: 0.8271 - accuracy: 0.5859 - val_loss: 0.8447 - val_accuracy: 0.4062
Epoch 2/20
4/4 [==============================] - 8s 2s/step - loss: 0.6985 - accuracy: 0.6406 - val_loss: 0.6913 - val_accuracy: 0.5625
Epoch 3/20
4/4 [==============================] - 8s 2s/step - loss: 0.6269 - accuracy: 0.6172 - val_loss: 0.5917 - val_accuracy: 0.6562
Epoch 4/20
4/4 [==============================] - 8s 2s/step - loss: 0.5479 - accuracy: 0.7266 - val_loss: 0.4991 - val_accuracy: 0.7500
Epoch 5/20
4/4 [==============================] - 8s 2s/step - loss: 0.4770 - accuracy: 0.7578 - val_loss: 0.4373 - val_accuracy: 0.8438
Epoch 6/20
4/4 [==============================] - 8s 2s/step - loss: 0.4046 - accuracy: 0.8125 - val_loss: 0.2979 - val_accuracy: 0.9062
Epoch 7/20
4/4 [==============================] - 8s 2s/step - loss: 0.3878 - accuracy: 0.8203 - val_loss: 0.3436 - val_accuracy: 0.8125
Epoch 8/20
4/4 [=========================

## Run with test data

In [10]:
pre = tf.keras.metrics.Precision()
re = tf.keras.metrics.Recall()
acc = tf.keras.metrics.BinaryAccuracy()

for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)
    
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}')

1/1 [==============================] - 0s 267ms/step
Precision: 1.0, Recall: 1.0, Accuracy: 1.0


## Save the model

 To test different brand new images that are not in the `data_dir`.

### .h5

In [11]:
model.save('model.h5')

### .tflite

In [12]:
# Convert the model to the TensorFlow Lite format

converter = tf.lite.TFLiteConverter.from_keras_model(model)
# # USE QUANTIZATION
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

# Save the model to disk
open("model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Bahillo\AppData\Local\Temp\tmp7r32hj4b\assets


INFO:tensorflow:Assets written to: C:\Users\Bahillo\AppData\Local\Temp\tmp7r32hj4b\assets


14790588

#### Optional
For edge devices, see [supported platforms](https://www.tensorflow.org/lite/microcontrollers#supported_platforms)

If the device requires to use C header file (.h) ...





In [13]:
def hex_to_c_array(hex_data) -> str:
    # Declare C variable
    c_str = 'unsigned char model[] = {'
    hex_array = []

    for i, val in enumerate(hex_data):
        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n'
        hex_array.append(hex_str)

    # Wrapping up
    c_str += '\n ' + format(' '.join(hex_array)) + '};'

    return c_str

with open('model.h', 'w') as f:
    content = hex_to_c_array(tflite_model)
    f.write(content)

## Test new images

### Load the model using the **.h5** file

In [19]:
loaded_model =  tf.keras.models.load_model('model.h5')

### Begin prediction testing

In [53]:
import numpy as np

threshold = 0.5

img = cv2.imread('happytest.jpg') # replace with your file name here
test_image = tf.image.resize(img, (256, 256))
yhat = model.predict(np.expand_dims(test_image/255, 0))

print('\n', yhat)

if yhat > 0.5:
    print('Predicted class is Sad')
else:
    print('Predicted class is Happy')

1/1 [==============================] - 0s 44ms/step

 [[0.00367736]]
Predicted class is Happy
